In [1]:
import re
import string
import pandas as pd
from math import log
import os
import random
import re
import string
from collections import Counter
from sklearn.cluster import KMeans

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

from transformers import AutoTokenizer, AutoModel
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

[nltk_data] Downloading package stopwords to /Users/joker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv(r"/Users/joker/Library/CloudStorage/OneDrive-IITKanpur/Thesis/data_100k.csv")
df = df.loc[:, ['description']]


In [3]:
def clean_text(text):
    text = str(text).lower() 
    text = re.sub(r"\[(.*?)\]", "", text)  
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text) 
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text) 
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation
    text = re.sub('[^a-z\s]+', '', text)
    return text

In [4]:
def freq(sent):
    arr = np.zeros(26)
    for letter in sent:
        if 97 <= ord(letter) <= 122:
            # print("here")
            arr[ord(letter) - 97] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

In [5]:
df['description_cleaned'] = df['description'].apply(clean_text)

In [6]:
df['histograms'] = df['description_cleaned'].apply(freq)

In [7]:
clusters = 2
def mbkmeans_clusters(X, k):
    km = KMeans(n_clusters=k).fit(X)
    return km.labels_

In [8]:
df = df.loc[ df.histograms.map(lambda x: ~np.isnan(x).any()), :]

In [9]:
cluster_labels = mbkmeans_clusters(
	X=list(df['histograms'].values),
    k=clusters,
)

In [10]:
df["cluster"] = cluster_labels

In [11]:
hin = df[df.cluster == 1]

In [12]:
hin

,description,description_cleaned,histograms,cluster
3,Sir mai virendra pratap singh uttar pradesh ka...,sir mai virendra pratap singh uttar pradesh ka...,"[0.6729258182937208, 0.08504007593821747, 0.02...",1
5,Respected Sir I am a B.tech student. I follows...,respected sir i am a btech student i follows y...,"[0.541687646512873, 0.07194289055249095, 0.084...",1
18,Shriman Narendra Modi ji Mera Naam Anil Kumar ...,shriman narendra modi ji mera naam anil kumar ...,"[0.6543680961547901, 0.07634294455139218, 0.06...",1
24,Seva Mein Shriman mananiy Pradhanmantri mahoda...,seva mein shriman mananiy pradhanmantri mahoda...,"[0.6189839237896101, 0.04609454751624756, 0.0,...",1
26,"Good evening sir,sir mere dwara ccsu meerut ma...",good evening sirsir mere dwara ccsu meerut mai...,"[0.5848016933547654, 0.06961924920890064, 0.08...",1
...,...,...,...,...
99979,Shri Pradhanmantri Narendra Modi ji Meri aapse...,shri pradhanmantri narendra modi ji meri aapse...,"[0.6979755597443041, 0.08211477173462402, 0.0,...",1
99980,Modi ji up me rojgar ka bhaout bura haal hai a...,modi ji up me rojgar ka bhaout bura haal hai a...,"[0.6294033330928512, 0.13112569439434402, 0.10...",1
99982,sar mein ek bahut chhote se gaon ka Rahane wal...,sar mein ek bahut chhote se gaon ka rahane wal...,"[0.6306570524869749, 0.07459384491781423, 0.10...",1
99988,Sirr plzz request aa ssc gd 2018 all medical f...,sirr plzz request aa ssc gd all medical fit j...,"[0.6945372265949069, 0.056313829183370835, 0.1...",1


In [13]:
def count_total_hashes(sent, tokenizer):
    tot = 0
    lst  = tokenizer.tokenize(sent)
    # print(lst)
    for word in lst:
        tot += word.count('#')
    return tot
        

In [62]:
def count_total_hashes_m2m(sent, tokenizer = tokenizer_m2m):
    tot = 0
    lst  = tokenizer.tokenize(sent)
    # print(lst)
    for i in range(len(lst)):
        t = list(lst[i])
        
        if t[0] == '▁':
            t.pop(0)
        else: 
            t[0] = '#'+'#'+t[0]
        lst[i] = ''.join(t)   
        
    # print(lst)
    for word in lst:
        tot += word.count('#')
    return tot

In [61]:
def count_hash_to_character_ratio(word):
    c_hash, c_char = 0,0
    for c in word:
        if c=='#':
            c_hash+=1
        elif 97<=ord(c)<=122:
            c_char+=1
    return round(c_hash/c_char, 4)

In [72]:
def calculate_hash_to_character_ratio_sent(sent, tokenizer):
    lst  = tokenizer.tokenize(sent)
    
    dct = {}
    for word in lst:
        try:
            dct[count_hash_to_character_ratio(word)]+=1
        except:
            dct[count_hash_to_character_ratio(word)] = 0
            dct[count_hash_to_character_ratio(word)]+=1
    
    return {round(k,2):v for k,v in sorted(dct.items(), key=lambda x: x[0], reverse=True)}, len(lst)
            
            
        

In [73]:
def calculate_hash_to_character_ratio_sent_m2m(sent, tokenizer):
    lst  = tokenizer.tokenize(sent)
    for i in range(len(lst)):
        t = list(lst[i])
        
        if t[0] == '▁':
            t.pop(0)
        else: 
            t[0] = '#'+'#'+t[0]
        lst[i] = ''.join(t)
    dct = {}
    for word in lst:
        try:
            dct[count_hash_to_character_ratio(word)]+=1
        except:
            dct[count_hash_to_character_ratio(word)] = 0
            dct[count_hash_to_character_ratio(word)]+=1
    
    return {round(k,2):v for k,v in sorted(dct.items(), key=lambda x: x[0], reverse=True)}, len(lst)
            
            
        

In [75]:
calculate_hash_to_character_ratio_sent_m2m(s, tokenizer_m2m)

({1.0: 7, 0.67: 4, 0.4: 2, 0.0: 37}, 50)

In [59]:
s = "sar mein ek bahut chhote se gaon ka rahane wala hun aur ek meri chhoti si kirana dukaan hai main usi par apni dukaan chalata hun aur maine digital india for all ka if liya tha aur"

In [64]:
tokenizer_salesken = AutoTokenizer.from_pretrained("salesken/similarity-eng-hin_latin")
tokenizer_hing_bert = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
tokenizer_hing_mbert = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
tokenizer_m2m = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en")

In [20]:
hindi_grievances = list(hin.description_cleaned.values)

In [21]:
len(hindi_grievances)

12091

In [27]:
hindi_grievances = [g for g in hindi_grievances if 100 <len(g.split()) < 512]

In [28]:
len(hindi_grievances)

4471

In [39]:
count_total_hashes(s, tokenizer_salesken)

10

In [29]:
hindi_grievances_20 = random.sample(hindi_grievances, 500)

In [56]:
s = hindi_grievances_20[0]

In [69]:
count_total_hashes_m2m(s)

26

In [79]:
mean_df = pd.DataFrame(columns=['#s', 2.0, 1.0, 0.0])
for sent in hindi_grievances_20:
    # print(sent)
    tmp = []
    l1, sz = calculate_hash_to_character_ratio_sent(sent, tokenizer_hing_bert)
    
    l1[99.99] = count_total_hashes(sent, tokenizer_hing_bert)
    # l1[199.99] = sz
    l1 = {k: v / sz for k, v in l1.items()}
    l1 = {k:v for k,v in sorted(l1.items(), key=lambda x: x[0], reverse=True)}
    tmp.append(l1)
    
    
    l2,sz = calculate_hash_to_character_ratio_sent(sent, tokenizer_salesken)
    
    l2[99.99] = count_total_hashes(sent, tokenizer_salesken)
    # l2[199.99] = sz
    l2 = {k: v / sz for k, v in l2.items()}
    l2 = {k:v for k,v in sorted(l2.items(), key=lambda x: x[0], reverse=True)}
    tmp.append(l2)
    
    l3, sz = calculate_hash_to_character_ratio_sent(sent, tokenizer_hing_mbert)
    
    l3[99.99] = count_total_hashes(sent, tokenizer_hing_mbert)
    # l3[199.99] =  sz
    l3 = {k: v / sz for k, v in l3.items()}
    l3 = {k:v for k,v in sorted(l3.items(), key=lambda x: x[0], reverse=True)}
    tmp.append(l3)
    
    l4, sz = calculate_hash_to_character_ratio_sent_m2m(sent, tokenizer_m2m)
    l4[99.99] = count_total_hashes_m2m(sent, tokenizer_m2m)
    # l4[99.99] = count_total_hashes_m2m(sent)
    l4 = {k: v / sz for k, v in l4.items()}
    l4 = {k:v for k,v in sorted(l4.items(), key=lambda x: x[0], reverse=True)}
    tmp.append(l4)
    
    
    df = pd.DataFrame(tmp)
    lst = list(df.columns)
    # lst[0] = "Total_tokens"
    lst[0] = "#s"
    df.columns = lst
    df.fillna(0, inplace=True)
    # print(df, '\n')
    # print(lst)
    df =  df[['#s', 2.0, 1.0, 0.5, 0.0]]
    if len(mean_df)==0:
        mean_df = df
    else:
        mean_df = mean_df.add(df)
    
    # print(df.columns)
    
mean_df = (mean_df/500).round(2)
    

In [81]:
mean_df.index = ["tokenizer_hing_bert", "tokenizer_salesken", "tokenizer_hing_mbert", "tokenizer_m2m"]

In [82]:
mean_df

,#s,2.0,1.0,0.5,0.0
tokenizer_hing_bert,0.82,0.13,0.17,0.02,0.59
tokenizer_salesken,0.58,0.10,0.09,0.03,0.71
tokenizer_hing_mbert,0.79,0.12,0.15,0.03,0.61
tokenizer_m2m,0.72,0.04,0.16,0.04,0.64


- #s should be minimum
- 2.0 ratio means tokens with single letters, so should be minimum
- 1.0 ratio should be minimum, 2 hashes and 2 letters in a token
- 0.5 ratio should be maximum, 2 hashes and 4 letters in a token
- 0.0 should be maximum as these are fully recognised tokens

## Entropy based approach

In [83]:
s

'sar mein ek bahut chhote se gaon ka rahane wala hun aur ek meri chhoti si kirana dukaan hai main usi par apni dukaan chalata hun aur maine digital india for all ka if liya tha aur'

In [1]:
tokenizer_salesken = AutoTokenizer.from_pretrained("salesken/similarity-eng-hin_latin")
tokenizer_hing_bert = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
tokenizer_hing_mbert = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
tokenizer_m2m = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en")

def count_chars(word):
    c = 0
    for w in word:
        if 97<=ord(w)<=122:
            c+=1
    return c

def count_freq(s, tokeniser):
    d = dict()
    lst = tokeniser.tokenize(s)
    counts = sorted([count_chars(w) for w in lst])
    return counts

def entropy(labels, base=None):

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value,counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  # Compute entropy
  base = np.e if base is None else base
  for i in probs:
    ent -= i * log(i, base)

  return ent


tokenizer_hing_bert_sum = 0
tokenizer_hing_mbert_sum = 0
tokenizer_m2m_sum = 0
tokenizer_salesken_sum = 0

n = len(hindi_grievances)

for sent in hindi_grievances:
    tokenizer_m2m_sum += entropy(count_freq(s, tokenizer_m2m), 2)
    tokenizer_hing_bert_sum += entropy(count_freq(s, tokenizer_hing_bert), 2)
    tokenizer_hing_mbert_sum += entropy(count_freq(s, tokenizer_hing_mbert), 2)
    tokenizer_salesken_sum += entropy(count_freq(s, tokenizer_salesken), 2)

out = dict()

out["tokenizer_hing_bert"] = tokenizer_hing_bert_sum/n
out["tokenizer_hing_mbert"] = tokenizer_hing_mbert_sum/n
out["tokenizer_m2m_sum"] = tokenizer_m2m_sum/n
out["tokenizer_salesken_sum"] = tokenizer_salesken_sum/n

out = {k:v for k,v in sorted(out.items(), key=lambda x: x[1])}



NameError: name 'AutoTokenizer' is not defined

1.9701380294277913
1.9941253738291251
2.025290531126644
2.2281789842251594


In [113]:
out

{'tokenizer_m2m_sum': 1.9701380294279907,
 'tokenizer_hing_bert': 1.9941253738290246,
 'tokenizer_hing_mbert': 2.025290531126495,
 'tokenizer_salesken_sum': 2.228178984225302}